In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import KFold

In [2]:
path = '/home/kai/data/kaggle/talkingdata/data/'
train = pd.read_csv(path + 'train_cleaned_final.csv')
test = pd.read_csv(path + 'test_cleaned_final.csv')

In [4]:
### load feature cols
import json as js
featurefile = '/home/kai/data/kaggle/talkingdata/wl/data/features/feature_cols.json'
with open(featurefile, 'r') as myjs:
    added_features = js.load(myjs)

# use last 65million as example

In [5]:
length = 75 * 1000000
front = len(train) - length
df_history = train.iloc[:front].copy()
df_train = train.iloc[front:].copy()
# last 65m + test
df_all = pd.concat([df_train, test])  

import gc
del train
del test
del df_train
gc.collect()

32

In [6]:
import sys
print(sys.getsizeof(df_all)/ 1024 **3)
print(len(df_all))

8.385521404445171
93790469


In [7]:
orders = {}
feature_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel',
              'day',
              'hour',
              'minute',
              'second']

# feature_col = ['ip', 
#               'app', 
#               'device', 
#               'os', 
#               'channel']
for col in feature_col:
    orders[col] = 10 ** (int(np.log(max(df_history[col].max(),df_all[col].max() ) + 1) / np.log(10)) + 1)
def get_group(df, cols):
    """
    define an encoding method which can ganrantee the adding value will be unique.
    eg: artist_name_composer will be a combination of (artist_name,composer) and the encoding will reflect the unqiue combination of those two
    """
    group = df[cols[0]].copy()
    for col in cols[1:]:
        group = group * orders[col] + df[col]
        
    return group

import gc
# del train
gc.collect()

41

In [8]:
orders

{'app': 1000,
 'channel': 1000,
 'day': 100,
 'device': 10000,
 'hour': 100,
 'ip': 1000000,
 'minute': 100,
 'os': 1000,
 'second': 100}

# count
plan 1. count from historical data  
plan 2. count from all data

In [9]:
def count(df_history, df_train, cols, target=None):
    """
    Purpose: add a new feature to training df.count the number of records for each feature combination (eg, artist_name_composer)) 
    """
    
    group = get_group(df_train, cols)
    group_all = get_group(df_history, cols)
    
    count_map = group_all.value_counts()
    
    return group.map(count_map).fillna(0)

# mean
mean P(target | feature combination)

purpose: add a new feature to training df. conditional probability P(replay (target) | feature combination (eg, artist_name_composer))
Get the conditional Probability only from historical data and apply to train data.

P(replay | X feature combination) = P( replay & X feature combination) / P (X feature combination)  
=(count(replay & X feature combination) / count(total)) / (count(X feature combination) / count(total))  
= count(replay & X feature combination) / count(X feature combination)  
= sum((replay & X feature combination)) / count(X feature combination)  
= sum((replay or not replayed & X feature combination)) / count(X feature combination)# since replay is 1, not replay is 0  
= sum( X feature combination) / count(X feature combination)  
= mean(X feature combination)  

In [10]:
def scaller(num):
    sca = 1
    while num * sca < 1:
        sca *= 10
    return sca

def mean(df_history, df_train, cols, target):
    """
    Purpose: add a new feature to training df. conditional probability P(replay (target) | feature combination (eg, artist_name_composer)) 
    """
  
    # encoding df's cols into a new series
    group = get_group(df_train, cols)
    # encoding df_history's cols into a new series
    group_history = get_group(df_history, cols)
    # get the conditional probability p(target| feature combination. eg, artist_name_composer) 
    mean_map = df_history.groupby(group_history)[target].mean()
    # mean_map: key - encoding, value - target mean
#     ### sca
#     m_min = mean_map[mean_map > 0].min()
#     sca = scaller(m_min)
#     mean_map *= sca
#     ###

    return group.map(mean_map).fillna(-1)

# reversemean
reverse mean P(feature combination | target)

In [11]:
def reversemean(df_history, df_train, cols, target):
    """
    Purpose: add a new feature to training df. conditional probability P(replay (target) | feature combination (eg, artist_name_composer)) 
    """
  
    # encoding df's cols into a new series
    group = get_group(df_train, cols)
    # encoding df_history's cols into a new series
    group_history = get_group(df_history, cols)
    # get the conditional probability p(target| feature combination. eg, artist_name_composer) 
    positive = group_history[df_history[target] == 1]
    negative = group_history[df_history[target] == 0]
    index_p = set(positive.unique())
    index_n = set(negative.unique())
    index_n.difference_update(index_p)
    map_reverse_p = positive.groupby(positive).count() / len(positive)
    map_reverse_n = pd.Series(np.zeros(len(index_n)), index=index_n)
    map_reverse = pd.concat([map_reverse_p, map_reverse_n])
    return group.map(map_reverse).fillna(-1)

# generate all cols

In [13]:
from itertools import combinations


combine_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel', 
              'day',
              'hour',
              'minute',
              'second']

# combine_col = ['ip', 
#               'app', 
#               'device', 
#               'os', 
#               'channel']

def col_name(cols, func=None):
    if func is None:
        return '_'.join(cols)
    else:
        return '_'.join(cols) + '_' + func.__name__

counter = 0



exception_list = []
for num_col in [1,2,3]:
    for cols in combinations(combine_col, num_col):
#         for func in [count, mean]:
        for func in [count, mean]:
            feature_name = col_name(cols, func=func)
            if feature_name in added_features:
                counter += 1
                if func.__name__ == count.__name__:
                    print('count function')
                    df_all[feature_name] = func(df_all, df_all, cols, target='is_attributed')
                else:
                    print('mean function')
                    df_all[feature_name] = func(df_history, df_all, cols, target='is_attributed')
                all_str = 'all {}:   {}   \t\t\t size: {} G.'.format(counter, feature_name, sys.getsizeof(df_all)/ 1024 **3)
                print(all_str)
                with open('feature_all.txt', 'w') as text_file:
                    text_file.write(all_str + '\n')


count function
all 1:   ip_count   			 size: 9.084314852952957 G.
mean function
all 2:   ip_mean   			 size: 9.783108301460743 G.
count function
all 3:   ip_app_count   			 size: 10.481901749968529 G.
mean function
all 4:   ip_app_mean   			 size: 11.180695198476315 G.
count function
all 5:   ip_device_count   			 size: 11.8794886469841 G.
count function
all 6:   ip_channel_count   			 size: 12.578282095491886 G.
mean function
all 7:   ip_channel_mean   			 size: 13.277075543999672 G.
count function
all 8:   ip_day_count   			 size: 13.975868992507458 G.
count function
all 9:   ip_hour_count   			 size: 14.674662441015244 G.
count function
all 10:   ip_minute_count   			 size: 15.37345588952303 G.
count function
all 11:   ip_second_count   			 size: 16.072249338030815 G.
mean function
all 12:   ip_second_mean   			 size: 16.7710427865386 G.
count function
all 13:   app_channel_count   			 size: 17.469836235046387 G.
mean function
all 14:   device_minute_mean   			 size: 18.168629683554

# Saving Files

In [ ]:
target='is_attributed'
train_col = added_features.copy()
train_col.append(target)
train = df_all.iloc[:length][train_col]
test = df_all.iloc[length:][added_features]

In [ ]:
train.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/train_countall_countAndmean_last75m.csv', index=False)
print('training saving done!')
test.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/test_countall_countAndmean_last75m.csv', index=False)
print('testing saving done!')

In [14]:
#For float saving


# df_train.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/train_fold_last_in_12_mean_1float.csv', index=False, float_format='%.1f')
# print('training saving done!')
# test.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/test_fold_last_in_12_mean_1float.csv', index=False, float_format='%.1f')
# print('testing saving done!')

training saving done!
testing saving done!


In [27]:
list(train.columns)

['ip_count',
 'ip_minute_count',
 'app_channel_count',
 'ip_app_count',
 'ip_app_hour_count',
 'ip_second_count',
 'ip_app_os_count',
 'ip_device_hour_count',
 'ip_os_hour_count',
 'ip_day_hour_count',
 'ip_channel_count',
 'app_channel_day_count',
 'ip_device_count',
 'ip_day_count',
 'ip_hour_count',
 'channel',
 'app',
 'os',
 'device',
 'second',
 'minute',
 'hour',
 'ip',
 'is_attributed']

In [31]:
list(test.columns)

['ip_count',
 'ip_minute_count',
 'app_channel_count',
 'ip_app_count',
 'ip_app_hour_count',
 'ip_second_count',
 'ip_app_os_count',
 'ip_device_hour_count',
 'ip_os_hour_count',
 'ip_day_hour_count',
 'ip_channel_count',
 'app_channel_day_count',
 'ip_device_count',
 'ip_day_count',
 'ip_hour_count',
 'channel',
 'app',
 'os',
 'device',
 'second',
 'minute',
 'hour',
 'ip']

In [32]:
test.head(10)

,ip_count,ip_minute_count,app_channel_count,ip_app_count,ip_app_hour_count,ip_second_count,ip_app_os_count,ip_device_hour_count,ip_os_hour_count,ip_day_hour_count,...,ip_day_count,ip_hour_count,channel,app,os,device,second,minute,hour,ip
0,506,8,900035,88,9,8,3,47,3,34,...,91,47,107,9,3,1,0,0,4,5744
1,10721,231,1831868,1335,90,155,19,818,40,403,...,2083,821,466,9,3,1,0,0,4,119901
2,6855,144,1138028,456,23,107,65,588,251,229,...,2135,588,128,21,19,1,0,0,4,72287
3,5470,136,291728,263,30,96,58,560,150,239,...,1201,560,111,15,13,1,0,0,4,78477
4,660,8,893563,90,9,11,28,82,48,60,...,208,83,328,12,13,1,0,0,4,123080
5,1874,33,4191211,172,21,40,41,239,69,120,...,399,239,107,18,13,1,0,0,4,110769
6,1934,31,1005191,344,27,27,3,154,6,90,...,412,159,137,3,1,1,0,0,4,12540
7,2784,27,398008,29,3,46,7,251,32,93,...,558,253,153,27,19,1,0,0,4,88637
8,3041,51,4191211,246,29,47,2,260,1,106,...,608,261,107,18,10,1,0,0,4,14932
9,13714,267,115820,1692,104,241,10,890,2,539,...,2718,1036,424,12,53,1,0,0,4,123701
